SNR Script (Power of first 10% length of audio as noise)

In [99]:
import os
import numpy as np
import pandas as pd
import soundfile as sf
from scipy.signal import stft
from scipy.signal.windows import hamming

def compute_snr(signal, noise, window_length=512, overlap=256):
    """
    Compute frame-wise SNR for a signal and noise using STFT.
    """
    window = hamming(window_length, sym=False)
    _, _, Zxx_signal = stft(signal, window=window, nperseg=window_length, noverlap=overlap)
    _, _, Zxx_noise = stft(noise, window=window, nperseg=window_length, noverlap=overlap)

    # Align dimensions
    min_frames = min(Zxx_signal.shape[1], Zxx_noise.shape[1])
    Zxx_signal = Zxx_signal[:, :min_frames]
    Zxx_noise = Zxx_noise[:, :min_frames]

    # Compute power
    signal_power = np.abs(Zxx_signal)**2
    noise_power = np.abs(Zxx_noise)**2

    # Compute SNR frame-wise
    snr = 10 * np.log10((signal_power + 1e-10) / (noise_power + 1e-10))  # Avoid log(0)
    mean_snr = np.mean(snr)
    std_snr = np.std(snr)

    return mean_snr, std_snr

def compute_snr_total(signal, noise):
    """
    Compute total SNR using the ratio of total signal power to noise power.
    """
    signal_power = np.sum(signal**2)
    noise_power = np.sum(noise**2) + 1e-10  # Avoid division by zero
    return 10 * np.log10(signal_power / noise_power)

# Define paths
unfiltered_directory = '/media/lab2208/New Volume/Organized Voice data HDD/Collection 6 to 17 Raw/Collection 8 Raw/noisy'
filtered_directory = '/media/lab2208/New Volume/Organized Voice data HDD/WaveUNet/full_length_filtered_16kHz/Collection 8'
output_excel = '/media/lab2208/New Volume/Organized Voice data HDD/SNR_Results/CD_8_WaveUNet2.xlsx'

# Initialize results
results = []

# Process unfiltered and filtered files
for unfiltered_file in os.listdir(unfiltered_directory):
    if unfiltered_file.endswith('.wav'):
        # Unfiltered file path
        unfiltered_path = os.path.join(unfiltered_directory, unfiltered_file)

        # Corresponding filtered file path
        filtered_path = os.path.join(filtered_directory, unfiltered_file)

        if os.path.exists(filtered_path):  # Ensure corresponding filtered file exists
            # Read unfiltered audio
            unfiltered_audio, sr = sf.read(unfiltered_path)
            if unfiltered_audio.ndim > 1:
                unfiltered_audio = np.mean(unfiltered_audio, axis=1)

            # Read filtered audio
            filtered_audio, _ = sf.read(filtered_path)
            if filtered_audio.ndim > 1:
                filtered_audio = np.mean(filtered_audio, axis=1)

            # Noise estimation: First 10% of the unfiltered signal
            noise = unfiltered_audio[:len(unfiltered_audio) // 10]

            # Zero-pad noise if shorter than signal
            if len(noise) < len(unfiltered_audio):
                noise = np.pad(noise, (0, len(unfiltered_audio) - len(noise)), mode='constant')

            # Compute SNR for unfiltered audio
            mean_snr_unfiltered, std_snr_unfiltered = compute_snr(unfiltered_audio, noise)
            total_snr_unfiltered = compute_snr_total(unfiltered_audio, noise)

            # Compute SNR for filtered audio
            mean_snr_filtered, std_snr_filtered = compute_snr(filtered_audio, noise)
            total_snr_filtered = compute_snr_total(filtered_audio, noise)

            # Append results
            results.append({
                'Subject': unfiltered_file,
                'Mean SNR (Unfiltered)': mean_snr_unfiltered,
                'Std SNR (Unfiltered)': std_snr_unfiltered,
                'Total SNR (Unfiltered)': total_snr_unfiltered,
                'Mean SNR (Filtered)': mean_snr_filtered,
                'Std SNR (Filtered)': std_snr_filtered,
                'Total SNR (Filtered)': total_snr_filtered
            })

# Save results to Excel
df = pd.DataFrame(results)
os.makedirs(os.path.dirname(output_excel), exist_ok=True)
df.to_excel(output_excel, index=False)

print(f"SNR comparison results saved to: {output_excel}")


SNR comparison results saved to: /media/lab2208/New Volume/Organized Voice data HDD/SNR_Results/CD_8_WaveUNet2.xlsx


The power of first 10ms/1s as noise

In [4]:
import os
import numpy as np
import pandas as pd
import soundfile as sf
from scipy.signal import stft
from scipy.signal.windows import hamming

def compute_snr(signal, noise, window_length=512, overlap=256):
    """
    Compute frame-wise SNR for a signal and noise using STFT.
    """
    window = hamming(window_length, sym=False)
    _, _, Zxx_signal = stft(signal, window=window, nperseg=window_length, noverlap=overlap)
    _, _, Zxx_noise = stft(noise, window=window, nperseg=window_length, noverlap=overlap)

    # Align dimensions
    min_frames = min(Zxx_signal.shape[1], Zxx_noise.shape[1])
    Zxx_signal = Zxx_signal[:, :min_frames]
    Zxx_noise = Zxx_noise[:, :min_frames]

    # Compute power
    signal_power = np.abs(Zxx_signal)**2
    noise_power = np.abs(Zxx_noise)**2

    # Compute SNR frame-wise
    snr = 10 * np.log10((signal_power + 1e-10) / (noise_power + 1e-10))  # Avoid log(0)
    mean_snr = np.mean(snr)
    std_snr = np.std(snr)

    return mean_snr, std_snr

def compute_snr_total(signal, noise):
    """
    Compute total SNR using the ratio of total signal power to noise power.
    """
    signal_power = np.sum(signal**2)
    noise_power = np.sum(noise**2) + 1e-10  # Avoid division by zero
    return 10 * np.log10(signal_power / noise_power)

# Define paths
unfiltered_directory = '/media/lab2208/New Volume/Organized Voice data HDD/Collection 6 to 17 Raw/Collection 8 Raw/noisy'
filtered_directory = '/media/lab2208/New Volume/Organized Voice data HDD/WaveUNet/full_length_filtered_16kHz/Collection 8'
output_excel = '/media/lab2208/New Volume/Organized Voice data HDD/SNR_Results/CD_8_WaveUNet_SNR_revised.xlsx'

# Initialize results
results = []

# Process unfiltered and filtered files
for unfiltered_file in os.listdir(unfiltered_directory):
    if unfiltered_file.endswith('.wav'):
        # Unfiltered file path
        unfiltered_path = os.path.join(unfiltered_directory, unfiltered_file)

        # Corresponding filtered file path
        filtered_path = os.path.join(filtered_directory, unfiltered_file)

        if os.path.exists(filtered_path):  # Ensure corresponding filtered file exists
            # Read unfiltered audio
            unfiltered_audio, sr = sf.read(unfiltered_path)
            if unfiltered_audio.ndim > 1:
                unfiltered_audio = np.mean(unfiltered_audio, axis=1)

            # Read filtered audio
            filtered_audio, _ = sf.read(filtered_path)
            if filtered_audio.ndim > 1:
                filtered_audio = np.mean(filtered_audio, axis=1)

            # Noise estimation: First 10 ms of the unfiltered signal
            noise_duration_ms = 2000  # Duration in milliseconds
            noise_samples = int(sr * noise_duration_ms / 1000)  # Number of samples for 10 ms

            noise = unfiltered_audio[:noise_samples]

            # Zero-pad noise if shorter than signal
            if len(noise) < len(unfiltered_audio):
                noise = np.pad(noise, (0, len(unfiltered_audio) - len(noise)), mode='constant')

            # Compute SNR for unfiltered audio
            mean_snr_unfiltered, std_snr_unfiltered = compute_snr(unfiltered_audio, noise)
            total_snr_unfiltered = compute_snr_total(unfiltered_audio, noise)

            # Compute SNR for filtered audio
            mean_snr_filtered, std_snr_filtered = compute_snr(filtered_audio, noise)
            total_snr_filtered = compute_snr_total(filtered_audio, noise)

            # Append results
            results.append({
                'Subject': unfiltered_file,
                'Mean SNR (Unfiltered)': mean_snr_unfiltered,
                'Std SNR (Unfiltered)': std_snr_unfiltered,
                'Total SNR (Unfiltered)': total_snr_unfiltered,
                'Mean SNR (Filtered)': mean_snr_filtered,
                'Std SNR (Filtered)': std_snr_filtered,
                'Total SNR (Filtered)': total_snr_filtered
            })

# Save results to Excel
df = pd.DataFrame(results)
os.makedirs(os.path.dirname(output_excel), exist_ok=True)
df.to_excel(output_excel, index=False)

print(f"SNR comparison results saved to: {output_excel}")


SNR comparison results saved to: /media/lab2208/New Volume/Organized Voice data HDD/SNR_Results/CD_8_WaveUNet_SNR_revised.xlsx
